In [ ]:
!{sys.executable} -m pip install openai

zsh:1: parse error near `-m'


In [ ]:
import pandas as pd
import json
from openai import OpenAI
client = OpenAI(api_key=/**/)

In [ ]:
system_prompt = """## INSTRUCTIONS ##

You are an expert fact checker. You will receive a new article. Read the article carefully, paying attention to any persons or organizations mentioned in the article. Then, return a valid JSON list of dictionaries (one per mentioned person or organization), each with the following fields:

- person_org: Name of any person or organisation mentioned by name in the article.,

- nationality: For each person or organisation, state their nationality (for example, Israel or Palestine, or any other nationality), if known. Infer their nationality from the rest of the text, if possible.

- description: For each person or organisation, add any other description, if any.

Do NOT include any other text or response besides the JSON. Your response MUST be machine-readable as a valid JSON list of dictionaries."""


In [ ]:
def query_article(text):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": text
            }
        ]
    )
    return response.choices[0].message.content

In [ ]:
df = pd.read_csv('./data/summary_20240421_articles.csv')
df['date'] = pd.to_datetime(df['date'], format='mixed')
root_dir = './'



#import each article
# convert txt to string
# run through query_article
# get output json
# append all output jsons into one
# save output json file

In [ ]:
json_list = []
invalid_jsons = []

# Loop through each article
for index, row in df.iterrows():
    if index not in [783, 829, 858, 862, 882, 891, 1098, 1182]:
        continue

    article_file = row['article_file']

    # Open original text block from preprocessed data file
    original_filename = root_dir + article_file
    f = open(original_filename, "r")
    article_text = f.read()
    f.close()

    query_output = query_article(article_text)

    if query_output:
        try:
            new_entries = json.loads(query_output)
            json_list.extend(new_entries)
        except json.JSONDecodeError:
            print("Query output is not valid JSON:", query_output)
            print("Index:", index)
            invalid_jsons.append(index)
            continue

    # Append to previous query outputs
    json_list = json_list + new_entries


# Convert the appended list back to JSON
json_output = json.dumps(json_list)

json_filepath = 'named_entities3.json'
with open(json_filepath, "w") as json_file:
    json_file.write(json_output)
print(invalid_jsons)

Query output is not valid JSON: ```json
[
    {
        "person_org": "José Andrés",
        "nationality": "Spain",
        "description": "Founder of World Central Kitchen (WCK)"
    },
    {
        "person_org": "Benjamin Netanyahu",
        "nationality": "Israel",
        "description": "Israeli Prime Minister"
    },
    {
        "person_org": "Erin Gore",
        "nationality": "Unknown",
        "description": "Chief Executive Officer of World Central Kitchen (WCK)"
    },
    {
        "person_org": "Joe Biden",
        "nationality": "United States",
        "description": "President of the United States"
    },
    {
        "person_org": "Karine Jean-Pierre",
        "nationality": "Unknown",
        "description": "White House press secretary"
    },
    {
        "person_org": "World Central Kitchen (WCK)",
        "nationality": "Unknown",
        "description": "Charity organization"
    },
    {
        "person_org": "Israel Defense Forces (IDF)",
        "nationalit

In [ ]:
PALESTINE_IDENTIFIERS = ["Palestine", "Palestinian", "Palestinians"]
ISRAEL_IDENTIFIERS = ["Israel", "Israeli", "Israelis"]
GAZA_IDENTIFIERS = ["Gaza", "Gazan", "Gazans"]
HAMAS_IDENTIFIERS = ["Hamas"]

# Cities in the West Bank and Gaza
PALESTINIAN_CITIES = ["Gaza", "Gaza Strip", "Jerusalem", "Abasan al-Kabira", "Abu Dis", "Bani Na\'im", "Bani Suheila",
                      "Beit Hanoun", "Beit Jala", "Beit Lahia", "Beit Sahour", "Beit Ummar", "Beitunia", "Bethlehem",
                      "Beit Lahm", "al-Bireh", "Deir al-Balah", "ad-Dhahiriya", "Dura", "Gaza City", "Ghazzah",
                      "Halhul",
                      "Hebron", "al-Khalil", "Idhna", "Jabalia", "Jenin", "Jericho", "Ariha", "Khan Yunis", "Nablus",
                      "Qabatiya", "Qalqilya", "Rafah", "Ramallah", "Sa\'ir", "as-Samu", "Surif", "Tubas", "Tulkarm",
                      "Ya\'bad", "al-Yamun", "Yatta", "az-Zawayda", "Nazareth", "Jaljulia", "Kafr Bara", "Kafr Qasim",
                      "Qalansawe", "Tayibe", "Tira", "Zemer", "Ar\'ara", "Baqa al-Gharbiyye", "al-Arian", "Basma",
                      "Jatt",
                      "Kafr Qara", "Ma\'ale Iron", "Meiser", "Umm al-Fahm", "Umm al-Qutuf", "Lod", "Ramla",
                      "Wadi Nisnas",
                      "Halisa", "Kababir", "Abbas", "Daliyat al-Karmel", "Ein Hawd", "Fureidis", "Ibtin", "Isfiya",
                      "Jisr az-Zarqa", "Khawaled", "Abu Ghosh", "Beit Jimal", "Ein Naqquba", "Ein Rafa", "Beit Hanina",
                      "Beit Safafa", "Jabel Mukaber", "Old City", "Ras al-Amud", "Sheikh Jarrah", "Shuafat", "Silwan",
                      "Sur Baher", "At-Tur", "Umm Tuba", "Wadi al-Joz", "al-Walaja", "Abu Qrenat", "Abu Talul",
                      "Ar\'arat an-Naqab", "Ateer", "al-Atrash", "Bir Hadaj", "Dhahiyah", "Drijat", "Ghazzah", "Hura",
                      "Kukhleh", "Kuseife", "Lakiya", "Makhul", "Mitnan", "Mulada", "Qasr al-Sir", "Rahat", "al-Sayyid",
                      "Shaqib al-Salam", "Tirabin al-Sana", "Tel as-Sabi", "Umm Batin", "Abu Sinan", "Arab al-Aramshe",
                      "Arab al-Subeih", "Arab al-Na\'im", "Arraba", "Basmat Tab\'un", "Beit Jann", "Bi\'ina",
                      "Bir al-Maksur", "Bu\'eine Nujeidat", "Buqei\'a", "Daburiyya", "Ed Dahi", "Deir al-Asad",
                      "Deir Hanna", "Dmeide", "Eilabun", "Ein al-Asad", "Ein Mahil", "Fassuta", "Hamaam", "Hamdon",
                      "Hurfeish", "Hussniyya", "I\'billin", "Iksal", "Ilut", "Jadeidi-Makr", "Jish", "Julis",
                      "Ka\'abiyye-Tabbash-Hajajre", "Kabul", "Kafr Kanna", "Kafr Manda", "Kafr Misr", "Kafr Yasif",
                      "Kamanneh", "Kaukab Abu al-Hija", "Kfar Kama", "Kisra-Sumei", "Maghar", "Majd al-Krum",
                      "Manshiya Zabda", "Mashhad", "Mazra\'a", "Mi\'ilya", "Muqeible", "Nahf", "Na\'ura", "Nazareth",
                      "Nein", "Rameh", "Ras al-Ein", "Rehaniya", "Reineh", "Rumana", "Rumat al-Heib", "Sajur",
                      "Sakhnin",
                      "Sallama", "Sandala", "Sha\'ab", "Shefa-\'Amr", "Sheikh Danun", "Shibli–Umm al-Ghanam", "Sulam",
                      "Suweid Hamira", "Tarshiha", "Tamra City", "Tamra Village", "Tuba-Zangariyye", "Tur\'an", "Uzeir",
                      "Yafa an-Naseriyye", "Yanuh-Jat", "Yarka", "Zarzir", "Bani Suheila", "Beit Hanoun", "Beit Lahiya",
                      "Deir al-Balah", "Jabalia", "Khan Yunis", "Rafah"]

# Cities in Israel ('48 lands)
ISRAELI_CITIES = ["Acre", "Afula", "Arad", "Arraba", "Ashdod", "Ashkelon", "Baqa al-Gharbiyye", "Bat Yam", "Beersheba",
                  "Beit She\'an", "Beit Shemesh", "Bnei Brak", "Dimona", "Eilat", "El\'ad", "Giv\'at Shmuel",
                  "Givatayim",
                  "Hadera", "Haifa", "Herzliya", "Hod HaSharon", "Holon", "Jerusalem", "Kafr Qasim", "Karmiel",
                  "Kfar Saba",
                  "Kfar Yona", "Kiryat Ata", "Kiryat Bialik", "Kiryat Gat", "Kiryat Malakhi", "Kiryat Motzkin",
                  "Kiryat Ono",
                  "Kiryat Shmona", "Kiryat Yam", "Lod", "Ma\'alot-Tarshiha", "Migdal HaEmek",
                  "Modi\'in-Maccabim-Re\'ut",
                  "Nahariya", "Nazareth", "Nesher", "Ness Ziona", "Netanya", "Netivot", "Nof HaGalil", "Ofakim",
                  "Or Akiva",
                  "Or Yehuda", "Petah Tikva", "Qalansawe", "Ra\'anana", "Rahat", "Ramat Gan", "Ramat HaSharon", "Ramla",
                  "Rehovot", "Rishon LeZion", "Rosh HaAyin", "Safed", "Sakhnin", "Sderot", "Shefa-\'Amr", "Tamra",
                  "Tayibe",
                  "Tel Aviv-Yafo", "Tel Aviv", "Tiberias", "Tira", "Tirat Carmel", "Umm al-Fahm", "Yavne",
                  "Yehud-Monosson",
                  "Yokneam Illit"]

PALESTINE_MEMBER_AFFILIATIONS = PALESTINE_IDENTIFIERS + PALESTINIAN_CITIES + GAZA_IDENTIFIERS
ISRAEL_MEMBER_AFFILIATIONS = ISRAEL_IDENTIFIERS + ISRAELI_CITIES

In [ ]:
def extract_affiliations(data, nationality_spellings):
    affiliations = set()
    for entry in data:
        if entry["nationality"]:
            if entry["nationality"].lower() in nationality_spellings:
                affiliations.add(entry["person_org"])
    return list(affiliations)

#open the json
# for entry in json
json_output

In [ ]:
with open('named_entities.json', 'r') as file:
    nlp_json = json.load(file)

PALESTINE_MEMBER_AFFILIATIONS += extract_affiliations(nlp_json, ["palestine", "palestinian", "gazan"])
ISRAEL_MEMBER_AFFILIATIONS += extract_affiliations(nlp_json, ["israel", "israeli"])

In [ ]:
with open("pal_affiliations.json", "w") as json_file:
    json.dump(PALESTINE_MEMBER_AFFILIATIONS, json_file)

with open("israel_affiliations.json", "w") as json_file:
    json.dump(ISRAEL_MEMBER_AFFILIATIONS, json_file)